In [1]:
INPUTS = ['production/puri/tiles']
OUTPUT = 'production/puri/modeloutput/footprints/'
REGION = 'puri'
MODELNAME = 'efficientnet-b6'
CHECKPOINT = 'buildingsegmentation'
IMAGESIZE = 512
SCALES = [1, 3]
DENSE = False

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask as buildingcrop
from rasterio import features

import shapely
from shapely.geometry import shape, Polygon, Point
from shapely.affinity import affine_transform

from skimage import io
from skimage import measure
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi

import numpy as np
import torch 
import torch.nn as nn
import torch.distributed as dist

import os 
from tqdm import tqdm
from pathlib import Path
import argparse
import cv2

import sys
sys.path.append('nn/')
#from nbs.nn.unet import UNet
from unet import UNet
from azureml.core.model import Model

In [4]:
torch.__version__

'1.11.0'

In [5]:
torch.cuda.is_available()

False

In [6]:
INPUT = Path(INPUTS[0])
OUTPUT = Path(OUTPUT)

In [7]:
region_tiles = INPUT
tile_paths = [str(p) for p in region_tiles.glob('*') if 'tif' in str(p)]

In [8]:
num_tiles = len(tile_paths)
print(f'Number of tiles to process {num_tiles}')

Number of tiles to process 180


# 2.1 Load Model

In [9]:
MODELNAME: 'efficientnet-b6'
CHECKPOINT: 'checkpoints/buildingsegmentation'
IMAGESIZE: 512
SCALE: [1, 3]
DENSE: False


In [10]:
torch.cuda.is_available()

False

In [11]:
if num_tiles:
    model = UNet(name=MODELNAME, pretrained=None)
    #model = nn.DataParallel(model)
    
    # if torch.cuda.is_available():
    #     model = model.cuda()
    # else:
    #     model = model.to('cpu')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    #checkpoint_path = Path(Model.get_model_path(CHECKPOINT, version=1))
    checkpoint_path = Path('checkpoints/buildingsegmentation')
    if checkpoint_path.exists():
        print("=> loading checkpoint '{}'".format(str(checkpoint_path)))
        checkpoint = torch.load(str(checkpoint_path), map_location='cpu')
        loaded_dict = checkpoint['state_dict']
        sd = model.state_dict()
        for k in model.state_dict():
            if k in loaded_dict:
                sd[k] = loaded_dict[k]
        
        loaded_dict = sd
        model.load_state_dict(loaded_dict)
        best_score = checkpoint['best_score']
        start_epoch = checkpoint['epoch']
        print("loaded checkpoint from '{}' (epoch {}, best_score {})"
                .format(CHECKPOINT, checkpoint['epoch'], checkpoint['best_score']))

=> loading checkpoint 'checkpoints/buildingsegmentation'
loaded checkpoint from 'buildingsegmentation' (epoch 92, best_score 80.36363636363636)


In [12]:
# if num_tiles:
#     model = UNet(name=MODELNAME, pretrained=None)
#     model = nn.DataParallel(model)
    
#     if torch.cuda.is_available():
#         model = model.cuda()

    
#     #checkpoint_path = Path(Model.get_model_path(CHECKPOINT, version=1))
#     checkpoint_path = Path('checkpoints/buildingsegmentation')
#     if checkpoint_path.exists():
#         print("=> loading checkpoint '{}'".format(str(checkpoint_path)))
#         checkpoint = torch.load(str(checkpoint_path), map_location='cpu')
#         loaded_dict = checkpoint['state_dict']
#         sd = model.state_dict()
#         for k in model.state_dict():
#             if k in loaded_dict:
#                 sd[k] = loaded_dict[k]
        
#         loaded_dict = sd
#         model.load_state_dict(loaded_dict)
#         best_score = checkpoint['best_score']
#         start_epoch = checkpoint['epoch']
#         print("loaded checkpoint from '{}' (epoch {}, best_score {})"
#                 .format(CHECKPOINT, checkpoint['epoch'], checkpoint['best_score']))

# 2.2 Run Prediction

In [13]:
polygons = [] #footprints geometry
values = [] #pixel values
footprints_per_scale = {} #this is to store the lower scale footprints anf check if it intersects with higher scale
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [14]:
model.eval()
print(f'Running prediction on {num_tiles}')

Running prediction on 180


In [15]:
def transform_coords(polygon, transform):
    return affine_transform(polygon, [transform.a, transform.b, 
                                      transform.d, transform.e, 
                                      transform.xoff, transform.yoff])

def check_intersection(base, polygon):
    intersection = base.intersects(polygon)
    return intersection.any()
    
    

In [16]:
#import pdb

In [17]:
count = 0
for SCALE in SCALES:
    for index in tqdm(range(num_tiles)):
        try:
            tile_path = tile_paths[index]
            tile_name = tile_path.split('/')[-1]

            with rio.open(tile_path, 'r') as ref:
                transform = ref.transform
                crs = ref.crs

                #read tile data
                data = ref.read()
                ref.close()
                data = np.transpose(data, (1, 2, 0))
                
                if DENSE:
                    data = data * 1.5
                
                h, w = (np.asarray(data.shape[:2]) * SCALE).astype('int32')
                scaled_image = cv2.resize(data, (w, h) , interpolation=cv2.INTER_NEAREST)
                scaled_mask = np.zeros((h, w, 3)).copy()
                for it in range(SCALE):
                    for jt in range(SCALE):
                        x = it * IMAGESIZE
                        y = jt * IMAGESIZE

                        image = scaled_image[y : y + IMAGESIZE, x : x + IMAGESIZE, :]
                        image = np.asarray(image, dtype='float32') / 255
                        for i in range(3):
                            image[..., i] = (image[..., i] - mean[i]) / std[i]

                        image = torch.from_numpy(image.transpose((2, 0, 1)).copy()).float()
                        image = image.unsqueeze(0)

                        with torch.no_grad():
                            #pdb.set_trace()
                            pred = model(image.to(device))
                            out = torch.sigmoid(pred).to(device).numpy()
                            out = out[0, ...]
                            out = np.transpose(out, (1, 2, 0))
                            scaled_mask[y : y + IMAGESIZE, x : x + IMAGESIZE, ...] = out

                mask = cv2.resize(scaled_mask, (IMAGESIZE, IMAGESIZE), interpolation=cv2.INTER_NEAREST)
                mask = mask[..., 2] * (1 -  mask[..., 1]) * (1 - mask[..., 0]) #subtract boundary and contact point
                
                #watershed method to seperate buildings
                mask0 = (mask > 0.6)
                local_maxi = peak_local_max(mask, indices=False, footprint=np.ones((11, 11)), 
                                                   labels=(mask > 0.4))
                local_maxi[mask0] = True
                seed_msk = ndi.label(local_maxi)[0]
                y_pred = watershed(-mask, seed_msk, mask=(mask > 0.4), watershed_line=True)
                y_pred = measure.label(y_pred, connectivity=2, background=0).astype('uint8')

                #generate polygons from the predictions
                polygon_generator = features.shapes(y_pred, mask=y_pred > 0)
                for polygon, value in polygon_generator:
                    poly = shape(polygon).buffer(0.0)
                    if poly.area > 5:
                        ############################################################################
                        #This entire logic is written because model does not do great job at scale 1 
                        #when image resolution is not good and there is high building density 
                        #so at scale the generate result will be considered when scale 1 fails to detect the buildinf
                        ############################################################################
                        #for first scale directly add the polygons
                        if SCALE != SCALES[-1]:
                            polygons.append(transform_coords(poly, transform))
                            values.append(value) 
                        else:
                            # check if building not detected already
                            base = footprints_per_scale[SCALES[0]]['geometry'] #this is footprints generated at first scale
                            if not check_intersection(base, poly):
                                polygons.append(transform_coords(poly, transform))
                                values.append(value)

                                                                                           
            footprints_per_scale[SCALE] = gpd.GeoDataFrame({'geometry': polygons})
        except Exception as e:
            print(e, index)
            continue

100%|███████████████████████████████████████████████████████████████████████████████| 180/180 [1:42:41<00:00, 34.23s/it]


# 2.3 Save Output

In [20]:
os.makedirs(str(OUTPUT), exist_ok=True)
if num_buildings:
    output_file = f'{str(OUTPUT)}/{REGION}_footprints_from_model.geojson'
    print(f'Writing output to {output_file}')
    polygon_gdf = gpd.GeoDataFrame({'geometry': polygons, 'value': values}, 
                                        crs=crs.to_wkt())
    polygon_gdf.to_file(f'{output_file}', driver='GeoJSON')
